In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
###Importing required libraries###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
data=pd.read_csv('../input/walmart-recruiting-trip-type-classification/train.csv.zip')
data.head(5)

In [ ]:
description=data['DepartmentDescription'].unique()###storing the different dept lists
weekday_grouping=data.groupby(['Weekday']).count()#Grouping by the different days of the week
y_axis=weekday_grouping.loc[:,'TripType']
x_axis=data['Weekday'].unique()
#print(weekday_grouping)
#print(weekday.shape)
plt.bar(x_axis,y_axis,color ='red',width = 0.4)
plt.show()

clearly the sales is high on the weekends and interestly on monday.

In [ ]:
Trip_type=data.groupby(['TripType']).count()#grouping by the TripType 
print(Trip_type)

In [ ]:
plt.figure(figsize = (12, 10))

sns.set_style('whitegrid')
ax = sns.countplot(x = 'TripType', data = data)
ax = ax.set(title = 'The Frequent of Trip Type', ylabel = 'Counts', xlabel = 'Trip Type')


clearly some triptype have more frequent visits

In [ ]:
print(data.isnull().sum())###checking for void values in the dataframe
data.dtypes
data['FinelineNumber'].unique()

In [ ]:
###Filling the void Values
data.loc[:,'DepartmentDescription'].fillna(value=999,inplace=True)
data.loc[:,'Upc'].fillna(value=0,inplace=True)
data.loc[:,'FinelineNumber'].fillna(value=0,inplace=True)
data.isnull().sum()


In [ ]:
df=pd.DataFrame()#defining a new dataframe 


In [ ]:
df['Count of items']=data.groupby(['VisitNumber'])['Upc'].count()#gives number of items purchased or returned by a customer
df.head(5)

In [ ]:
df['item_sum'] = data.groupby(['VisitNumber'])['ScanCount'].sum()#Net item purchased by a person
df.head(5)

In [ ]:
#Adding aditional features like weekday and TripType to the above dataframe
df = df.reset_index().merge(data[['TripType', 'VisitNumber', 'Weekday']].drop_duplicates(), on='VisitNumber')
df.head(5)

In [ ]:
###hot encoding the department description so that multiple rows of single customer can be converted to single row 
hot_encoding=pd.get_dummies(data,columns=['DepartmentDescription'])
hot_encoding.head(5)

In [ ]:

hot_encoding=hot_encoding.groupby(['VisitNumber']).sum()
hot_encoding.head(5)

In [ ]:
#Merging the hot encoded dataframe with the single_customer daatframe
x=pd.merge(left=df,right=hot_encoding,on='VisitNumber')
x.head(5)

In [ ]:
x.drop(columns=['TripType_y','Upc'],inplace=True)#duplicate columns
x.head(5)

In [ ]:
#label_encoding weekdays 
label_encoder= preprocessing.LabelEncoder()
x['Weekday']=label_encoder.fit_transform(x['Weekday'])
x.head(5)

In [ ]:
x.dtypes#checking for any string or object data type

In [ ]:
#calculating correction between the different features
corr=x.corr()
corr.head(5)

In [ ]:
heat_map=corr.iloc[0:7,0:7]
heat_map.head(5)

Here , the correlation with the single feature from hot encoded Departmental Description feature will be very less but the aggregate of all of them constituent a important feature,So correlation is not checked for them.

But for other important features correlation is checked to drop unimportant features

In [ ]:
sns.heatmap(heat_map,annot=True,fmt='.3f')


Here visit Number doesnt correlate much with the trip type which is expected as it acts as a index or id and is different for everyone ,also weekday has very less correlation with triptype,rest all the features correlates with the data in the similar magnitude.

In [ ]:
y=x.loc[:,'TripType_x']#taking out the target feature
print(y)

In [ ]:
x.drop(columns=['VisitNumber','TripType_x','Weekday'],inplace=True)#dropping unimportant features as well as the target feature
x.head(5)

In [ ]:
print(type(x))

In [ ]:
#standardising the data before implementing any model
from sklearn.preprocessing import StandardScaler  
st= StandardScaler()  
x= st.fit_transform(x)

In [ ]:
#spliting the data into test and train 
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=0)  
print(type(x_train))


In [ ]:
#Using logistic regression first
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression(max_iter=200,solver='liblinear')



In [ ]:

lg.fit(x_train,y_train)


In [ ]:
y_pred=lg.predict(x_test)


In [ ]:
print(y_pred)

In [ ]:
#calculating log loss for logistic regression
pred_lg = lg.predict_proba(x_test)
loss_lg = log_loss(y_test,pred) 
print(loss_lg)

In [ ]:
#calculating the accuracy of classification
accuray_lg=accuracy_score(y_pred,y_test)
print(accuray_lg*100)

In [ ]:
#applying RandomForest as a model
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)

In [ ]:
y_pred=rfc.predict(x_test)

In [ ]:
accuracy_rfc=accuracy_score(y_pred,y_test)
print(accuracy*100)

In [ ]:
y_axis=np.array([[accuray_lg,accuracy_rfc]])
y_axis=y_axis.T
models=['Logistic Regression','RandomForest']
models=list(models)

In [ ]:
plt.scatter(models,y_axis)
plt.show()

The logistic Regression has a less accuracy of 0.4 while Random forest has performed expectationally with accuracy of more than 0.99

In [ ]:
pred_rfc=rfc.predict_proba(x_test)
#print(pred_rfc)

loss_rfc=log_loss(y_test,pred_rfc)
print(loss_rfc)

In [ ]:
loss=np.array([[loss_lg,loss_rfc]]).T
plt.scatter(models,loss)


The RandomForestClassifer also has less log_loss than Logistic Regression (almost 10 times less)